In [1]:
import sys
import os
sys.path.append('..')
# from src.dataset import ImageDataLoader
# from src import config 

import torch
from pathlib import Path
from src.model import Net
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from PIL import Image

import torch
import torch.nn as nn
from torch.utils import data
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np


import cv2
import numpy as np
from torchvision import models
from efficientnet_pytorch import EfficientNet
from grad_cam import GradCam,GuidedBackpropReLUModel,show_cams,show_gbs,preprocess_image

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


In [2]:
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

In [3]:
def load_model_weights(model, model_path):
    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint['state_dict'])
    return model

In [4]:
hiper_path = '../artifacts/hiper_others_2023-05-12-18-06-48/4_fold_max_acc_checkpoint.pth.tar'
membran_path = '../artifacts/membran_others_2023-05-12-20-31-18/0_fold_max_acc_checkpoint.pth.tar'
sclero_path = '../artifacts/sclero_others_2023-05-12-13-27-54/4_fold_max_acc_checkpoint.pth.tar'
normal_path = '../artifacts/normal_others_2023-05-12-15-45-47/4_fold_max_acc_checkpoint.pth.tar'



hiper_model = load_model_weights(Net(net_version="b0", num_classes=2).to(DEVICE), hiper_path)
membran_model = load_model_weights(Net(net_version="b0", num_classes=2).to(DEVICE), membran_path)
sclero_model = load_model_weights(Net(net_version="b0", num_classes=2).to(DEVICE), sclero_path)
normal_model = load_model_weights(Net(net_version="b0", num_classes=2).to(DEVICE), normal_path)




Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0


In [8]:
hiper_model.backbone._conv_head

Conv2dStaticSamePadding(
  320, 1280, kernel_size=(1, 1), stride=(1, 1), bias=False
  (static_padding): Identity()
)

In [21]:
model = hiper_model
target_layers = [model.backbone._conv_head]
img = cv2.imread('../data/raw/hipercellularity/PAS/PSHIPERCELULARIDADE20200802-1394.jpg', 1)
img = np.float32(cv2.resize(img, (224, 224))) / 255
input_tensor = torch.from_numpy(img)
input_tensor = input_tensor.reshape(3, 224, 224)
# input_tensor = # Create an input tensor image for your model..
# Note: input_tensor can be a batch tensor with several images!

# Construct the CAM object once, and then re-use it on many images:
cam = GradCAM(model=model, target_layers=target_layers, use_cuda=True)

# You can also use it within a with statement, to make sure it is freed,
# In case you need to re-create it inside an outer loop:
# with GradCAM(model=model, target_layers=target_layers, use_cuda=args.use_cuda) as cam:
#   ...

# We have to specify the target we want to generate
# the Class Activation Maps for.
# If targets is None, the highest scoring category
# will be used for every image in the batch.
# Here we use ClassifierOutputTarget, but you can define your own custom targets
# That are, for example, combinations of categories, or specific outputs in a non standard model.

targets = [ClassifierOutputTarget(281)]

# You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

# In this example grayscale_cam has only one image in the batch:
grayscale_cam = grayscale_cam[0, :]
visualization = show_cam_on_image(input_tensor, grayscale_cam, use_rgb=True)

ValueError: expected 4D input (got 3D input)

In [17]:
input_tensor.unsqueeze(0).shape

torch.Size([1, 3, 224, 224])

In [4]:
# .backbone._blocks[1]
# hiper_model.backbone._modules.items()
hiper_model.to('cpu')

In [15]:

hiper_model = load_model_weights(Net(net_version="b0", num_classes=2).to('cpu'), hiper_path)

grad_cam = GradCam(model=hiper_model, blob_name = '_blocks', target_layer_names=[str(i) for i in range(1,16)], use_cuda=False)
img = cv2.imread('../data/raw/hipercellularity/PAS/PSHIPERCELULARIDADE20200802-1394.jpg', 1)
img = np.float32(cv2.resize(img, (224, 224))) / 255
inputs = preprocess_image(img)
# If None, returns the map for the highest scoring category.
# Otherwise, targets the requested index.
target_index = None
mask_dic = grad_cam(inputs, target_index)
show_cams(img, mask_dic)
gb_model = GuidedBackpropReLUModel(model=hiper_model, activation_layer_name = 'MemoryEfficientSwish', use_cuda=False)
show_gbs(inputs, gb_model, target_index, mask_dic)


Loaded pretrained weights for efficientnet-b0
[tensor([[[[-1.5374e+01, -1.6416e+01,  3.6840e+00,  ...,  9.9752e+00,
            2.5049e+01,  1.2364e+01],
          [ 7.2272e-01,  5.2604e-01, -6.2175e+00,  ...,  6.9073e+00,
            4.2774e+00, -6.1703e+00],
          [ 1.2355e+01, -7.5784e+00, -1.2004e+01,  ..., -1.3154e+01,
            1.5170e+01,  2.6351e+01],
          ...,
          [-3.7182e+00,  2.2605e+00, -2.2916e-01,  ..., -7.7778e+00,
            1.6496e+00,  1.8565e+00],
          [ 5.3102e-02, -1.2258e+00, -2.3844e+00,  ..., -4.3670e+00,
            8.3668e-01,  1.5869e+00],
          [-3.6608e+01, -4.1236e+01, -3.9795e+01,  ..., -4.9284e+01,
           -5.2839e+01, -3.2216e+01]],

         [[ 2.8277e+01,  2.5345e+01,  2.9307e+01,  ..., -9.5858e-01,
           -8.6768e+00, -1.1813e+01],
          [ 1.5263e+01,  6.1263e+00,  1.5278e+01,  ...,  4.7722e+00,
           -2.1145e+00, -2.3680e+00],
          [-1.0148e+01, -1.3872e+00, -2.9319e+00,  ...,  6.3977e+00,
           

In [8]:
inputs.shape

torch.Size([1, 3, 224, 224])

In [5]:

gradients =[]
outputs = []
def save_gradient( grad):
    gradients.append(grad)
x = inputs
blob_name ='_blocks'

for idx, module in hiper_model._modules.items():
        if idx != blob_name:
            try:
                x = module(x)
            except:
                x = x.view(x.size(0), -1)
                x = module(x)
        else:
            for name, block in enumerate(getattr(hiper_model,blob_name)):
                x = block(x)
                if str(name) in ['1','10','15']:
                    x.register_hook(save_gradient)
                    outputs += [x] 

NameError: name 'inputs' is not defined

In [13]:
for name, block in enumerate(getattr(hiper_model,blob_name)):
    print(block)

AttributeError: 'Net' object has no attribute '_blocks'

In [22]:
hiper_model._modules.items()
hiper_model.backbone

TypeError: 'EfficientNet' object is not subscriptable

In [6]:
import plotly.io as pio
pio.renderers.default = "png"

import json
import torch
from torchvision import models, transforms
from PIL import Image as PilImage

from omnixai.data.image import Image
from omnixai.explainers.vision.specific.gradcam.pytorch.gradcam import GradCAM

c:\Users\Maods\.pyenv\pyenv-win\versions\3.10.4\lib\site-packages\tqdm\auto.py:22: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

c:\Users\Maods\.pyenv\pyenv-win\versions\3.10.4\lib\site-packages\shap\utils\_clustering.py:35: NumbaDeprecationWarning:

The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.

c:\Users\Maods\.pyenv\pyenv-win\versions\3.10.4\lib\site-packages\shap\utils\_clustering.py:54: NumbaDeprecationWarning:

The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See

In [2]:
img = Image(PilImage.open('../data/raw/hipercellularity/PAS/2003PC0154af.jpg').convert('RGB'))

In [3]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
preprocess = lambda ims: torch.stack([transform(im.to_pil()) for im in ims])

In [5]:
explainer = GradCAM(
    model=model,
    target_layer=model._blocks[4],
    preprocess_function=preprocess
)
# Explain the top label
explanations = explainer.explain(img)
explanations.ipython_plot()

Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.


In [9]:
hiper_model.backbone._blocks[1]

MBConvBlock(
  (_expand_conv): Conv2dStaticSamePadding(
    16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False
    (static_padding): Identity()
  )
  (_bn0): BatchNorm2d(96, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_depthwise_conv): Conv2dStaticSamePadding(
    96, 96, kernel_size=(3, 3), stride=[2, 2], groups=96, bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn1): BatchNorm2d(96, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_se_reduce): Conv2dStaticSamePadding(
    96, 4, kernel_size=(1, 1), stride=(1, 1)
    (static_padding): Identity()
  )
  (_se_expand): Conv2dStaticSamePadding(
    4, 96, kernel_size=(1, 1), stride=(1, 1)
    (static_padding): Identity()
  )
  (_project_conv): Conv2dStaticSamePadding(
    96, 24, kernel_size=(1, 1), stride=(1, 1), bias=False
    (static_padding): Identity()
  )
  (_bn2): BatchNorm2d(24, eps=0.001, momentum=0.010000000000000009, affine=True, trac

In [23]:
vars(hiper_model)

{'training': True,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_pre_hooks': OrderedDict(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_hooks_with_kwargs': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_forward_pre_hooks_with_kwargs': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_state_dict_pre_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_load_state_dict_post_hooks': OrderedDict(),
 '_modules': OrderedDict([('backbone',
               EfficientNet(
                 (_conv_stem): Conv2dStaticSamePadding(
                   3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
                   (static_padding): ZeroPad2d((0, 1, 0, 1))
                 )
                 (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
                 (_blocks): ModuleList(